In [1]:
from openai import OpenAI
from langgraph.func import task, entrypoint
from typing_extensions import Literal
import pandas as pd
from langgraph.graph import StateGraph, END, MessagesState
from langgraph.types import Command
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from pydantic import BaseModel, Field

In [2]:
client = OpenAI(
    api_key="DeepSeek API Key",
    base_url="https://api.deepseek.com"
)

def call_deepseek(prompt: str, system_prompt: str = "You are a helpful assistant") -> str:
    response = client.chat.completions.create(
        model="deepseek-reasoner",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt},
        ],
        stream=False
    )
    return response.choices[0].message.content

In [3]:
from google.colab import files

uploaded = files.upload()

Saving short.txt to short (1).txt


In [3]:
with open("short.txt", "r", encoding="utf-8") as f:
    short_text = f.read()

short_text

'Record ID,Machine ID,Date,Description,Status\nR001,M08,2024-04-01,Routine inspection found no anomalies.,Pass\nR002,M03,2024-08-06,Equipment checked and operating within normal parameters.,Pass\nR003,M03,2024-12-18,Routine lubrication performed; all moving parts functional.,Pass\nR004,M05,2025-02-17,System performance verified; all readings within acceptable limits.,Pass\nR005,M09,2024-07-01,System performance verified; all readings within acceptable limits.,Pass\nR006,M04,2024-07-19,System performance verified; all readings within acceptable limits.,Pass\nR007,M08,2024-03-11,Hydraulic pressure levels consistent; system functioning correctly.,Pass\nR008,M07,2024-11-28,Routine lubrication performed; all moving parts functional.,Pass\nR009,M02,2024-02-04,"Voltage and current levels stable, no electrical faults detected.",Pass\nR010,M06,2024-01-25,"Voltage and current levels stable, no electrical faults detected.",Pass\nR011,M05,2024-07-23,Routine lubrication performed; all moving parts 

In [4]:
class Route(BaseModel):
    step: Literal["slope_calculation", "threshold_exceedance", "separate", "text_analysis"] = Field(...)

In [5]:
def router_model(user_msg: str) -> str:
    prompt = f"""
You're a routing controller. Categorize the input into ONE of the following:
- slope_calculation
- threshold_exceedance
- separate
- text_analysis

Only return the keyword. Do not explain.

Input: {user_msg}
"""
    response = client.chat.completions.create(
        model="deepseek-reasoner",
        messages=[
            {"role": "system", "content": "You are a strict intent classifier."},
            {"role": "user", "content": prompt}
        ],
        stream=False
    )
    return response.choices[0].message.content.strip().lower()

def supervisor_node(state: MessagesState) -> Command:
    user_msg = state["messages"][-1].content
    step = router_model(user_msg)
    return Command(update={"messages": state["messages"]}, goto=step)

In [13]:
def text_analysis_node(state: MessagesState) -> Command:
    system_prompt = "You are a maintenance report analyst. Your job is to classify maintenance logs provided in natural language."

    user_prompt = f"""
You are asked to classify each maintenance record as either pass or completed based on the description.
A pass record means that there is no issue for the machine and thus no maintenance required.
On the other hand, a completed record means there are anomalies or issues for the machine and thus maintenance is required.
Please only output the classification results. Do not explain.

---
{short_text}
"""

    response = client.chat.completions.create(
        model="deepseek-reasoner",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        stream=False
    )

    output = response.choices[0].message.content

    return Command(
        update={"messages": state["messages"] + [AIMessage(content=output)]},
        goto="end"
    )

In [7]:
def slope_agent_node(state: MessagesState) -> Command:
    return Command(update={"messages": state["messages"] + [AIMessage(content="[Slope] Slope calculation executed.")]}, goto="end")

def threshold_agent_node(state: MessagesState) -> Command:
    return Command(update={"messages": state["messages"] + [AIMessage(content="[Threshold] Threshold analysis executed.")]}, goto="end")

def separate_node(state: MessagesState) -> Command:
    return Command(update={"messages": state["messages"] + [AIMessage(content="[Separate] Prompts split handled.")]}, goto="end")

In [8]:
builder = StateGraph(MessagesState)

builder.add_node("supervisor", supervisor_node)
builder.add_node("slope_calculation", slope_agent_node)
builder.add_node("threshold_exceedance", threshold_agent_node)
builder.add_node("separate", separate_node)
builder.add_node("text_analysis", text_analysis_node)

builder.set_entry_point("supervisor")
builder.add_edge("slope_calculation", END)
builder.add_edge("threshold_exceedance", END)
builder.add_edge("separate", END)
builder.add_edge("text_analysis", END)

graph = builder.compile()

In [14]:
from langchain_core.messages import HumanMessage

test_prompt = "Please classify the uploaded maintenance log for me."
step = router_model(test_prompt)
initial_state = {
    "messages": [HumanMessage(content=test_prompt)]
}

print("Streaming begins...\n")
result = graph.invoke(initial_state)
print("Final Answer:\n")
print(result["messages"][-1].content)

Streaming begins...

Final Answer:

**Maintenance Log Summary**

**Overview:**  
- **Total Entries:** 100 (90 "Pass" routine checks, 10 "Completed" corrective actions).  
- **Machines Covered:** M01 to M10.  
- **Timeframe:** January 2024 – February 2025.  

---

### **Routine Maintenance (Pass Status)**  
1. **Common Tasks:**  
   - **Inspections:** Routine checks found no anomalies (e.g., R001, R012).  
   - **Lubrication:** Moving parts maintained (e.g., R003, R040).  
   - **Performance Verification:** Systems operating within acceptable limits (e.g., R004, R088).  
   - **Electrical Checks:** Stable voltage/current levels (e.g., R009, R057).  
   - **Hydraulic Systems:** Pressure levels consistent (e.g., R007, R050).  
   - **Conveyor Belts:** Tension verified (e.g., R013, R075).  
   - **Bearing Checks:** Noise levels monitored (e.g., R018, R090).  
   - **Safety Checks:** No issues detected (e.g., R019, R067).  

2. **Frequency:**  
   - Regular checks across all machines, with 